In [160]:
import numpy as np

In [211]:
class LinearModel:
    
    def __init__(self, A = np.empty([0,0]), b = np.empty([0,0]), c = np.empty([0,0]), minmax = "MAX"):
        self.A = A
        self.b = b
        self.c = c
        self.x = [float(0)] * len(c)
        self.minmax = minmax
        self.printIter = True
        self.optimalValue = None
        self.transform = False
        self.feasible = True
        self.bounded = False
        
    def addA(self, A):
        self.A = A
        
    def addB(self, b):
        self.b = b
        
    def addC(self, c):
        self.c = c
        self.transform = False
    
    def setObj(self, minmax):
        if(minmax == "MIN" or minmax == "MAX"):
            self.minmax = minmax
        else:
            print("Invalid objective.")
        self.transform = False
            
    def setPrintIter(self, printIter):
        self.printIter = printIter
            
    def printSoln(self):
        if(self.feasible):
            if(self.bounded):
                print("Coefficients: ")
                print(self.x)
                print("Optimal value: ")
                print(self.optimalValue)
            else:
                print("Problem Unbounded; No Solution")
        else:
            print("Problem Infeasible; No Solution")
        
    def printTableau(self, tableau):
        
        print("ind \t\t", end = "")
        for j in range(0, len(c)):
            print("x_" + str(j), end = "\t")
        for j in range(0, (len(tableau[0]) - len(c) - 2)):
            print("s_" + str(j), end = "\t")
        
        print()
        for j in range(0, len(tableau)):
            for i in range(0, len(tableau[0])):
                if(not np.isnan(tableau[j, i])):
                    if(i == 0):
                        print(int(tableau[j, i]), end = "\t")
                    else:
                        print(round(tableau[j, i], 2), end = "\t")
                else:
                    print(end = "\t")
            print()
            
    def getTableauPhase1(self):
        
        # construct starting tableau
        
        numVar = len(self.c)
        numArtificial = len(self.A)
        
        t1 = np.hstack(([None], [0], [0] * numVar, [0] * numArtificial))
                    
        basis = np.array([0] * numArtificial)
        
        for i in range(0, len(basis)):
            basis[i] = numVar + i
        
        A = self.A
        
        if(not ((numSlacArtificial + numVar) == len(self.A[0]))):
            B = np.identity(numArtificial)
            A = np.hstack((self.A, B))
            
        t2 = np.hstack((np.transpose([basis]), np.transpose([self.b]), A))
        
        tableau = np.vstack((t1, t2))
        
        for i in range(1, len(tableau[0]) - numArtificial):
            for j in range(1, len(tableau)):
                if(self.minmax == "MIN"):
                    tableau[0, i] -= tableau[j, i]
                else:
                    tableau[0, i] += tableau[j, i]
        
        tableau = np.array(tableau, dtype ='float')
        
        return tableau
    
    def driveOutAV(self, tableau):
        avPresent = False
        avInd = []
        
        #remove slack variables
        tableau = np.delete(tableau, obj = np.s_[2 + len(self.c):], axis = 1)
        
        #redundant row removal
        for i in range(1, len(tableau)):
            if tableau[i, 0] > len(self.c) - 1:
                
                redundant = True
                for j in range(2, len(self.c) + 2):
                    if (tableau[i, j] != 0):
                        redundant = False
                        avInd.append([i, False])
                        break
                avInd.append([i, redundant])
                
        if(len(avInd) == 0):
            print("\nNo artificial variables to drive out\n")
            return tableau
        
        for key in avInd:
            index, redundant = key
            if(redundant):
                tableau = np.delete(tableau, obj = index, axis = 0)
                avInd.remove(key)
            else:
                n = -1
                for i in range(2, len(tableau[0]) - 1):
                    if (tableau[r, i] != 0):
                        n = i
                        break
                tableau = pivot(tableau, index, n)
        
        return tableau
    
    def getTableauPhase2(self, tableau):
        
        for i in range(0, len(self.c) - 1):
            tableau[0, i + 2] = self.c[i]
            
        tableau[0, 1] = 0
        
        a = np.empty(0)
        for value in tableau[0, 1:]:
            a = np.append(a, value)
        
        for j in range(1, len(tableau)):
            for i in range(1, len(tableau[0])):
                if(self.minmax == "MIN"):
                    a[i - 1] -= tableau[0, int(tableau[j, 0] + 2)] * tableau[j, i]
                else:
                    a[i - 1] += tableau[0, int(tableau[j, 0] + 2)] * tableau[j, i]
            
        tableau[0, 1:] = a
            
        return tableau
    
    def pivot(self, tableau, r, n):
        
        pivot = tableau[r, n]
        
        # perform row operations 
        # divide the pivot row with the pivot element 
        tableau[r, 1:] = tableau[r, 1:] / pivot 
            
        # pivot other rows
        for i in range(0, len(tableau)): 
            if i != r:
                mult = tableau[i, n] / tableau[r, n]
                tableau[i, 1:] = tableau[i, 1:] - mult * tableau[r, 1:]
                
        return tableau
        
              
    def optimize(self):
        
        #if(self.minmax == "MIN" and self.transform == False):
            #for i in range(len(self.c)):
                #self.c[i] = -1 * self.c[i]
                #transform = True
        
        tableau = self.getTableauPhase1()
        
        print("Phase 1:\n")
        
        tableau = self.simplex(tableau)
        
        if(not self.bounded):
            return
        
        if(tableau[0, 1] != 0):
            self.feasible = False
            print("Problem Infeasible; No Solution")
            return
        
        print("Phase 2:\n")
        
        tableau = self.driveOutAV(tableau)
        
        tableau = self.getTableauPhase2(tableau)
        
        tableau = self.simplex(tableau)
        
        if(not self.bounded):
            return
       
        self.x = np.array([0] * len(c), dtype = float)
        
        # save coefficients
        for key in range(1, (len(tableau))):
            if(tableau[key, 0] < len(c)):
                self.x[int(tableau[key, 0])] = tableau[key, 1]
                
        self.optimalValue = -1 * tableau[0,1]

        
    def simplex(self, tableau):
        
        if(self.printIter == True):
            print("Starting Tableau:")
            self.printTableau(tableau)
        
        # assume initial basis is not optimal, problem is feasible, and problem is bounded
        optimal = False
        feasible = True
        bounded = True

        # keep track of iterations for display
        iter = 1

        while(True):
            
            if(self.printIter == True):
                print("----------------------------------")
                print("Iteration :", iter)
                self.printTableau(tableau)
                
            if(self.minmax == "MAX"):
                for profit in tableau[0, 2:]:
                    if profit > 0:
                        optimal = False
                        break
                    optimal = True
            else:
                for cost in tableau[0, 2:]:
                    if cost < 0:
                        optimal = False
                        break
                    optimal = True

            # if all directions result in decreased profit or increased cost
            if optimal == True: 
                 break
            
            # nth variable enters basis, account for tableau indexing
            if (self.minmax == "MAX"):
                n = tableau[0, 2:].tolist().index(np.amax(tableau[0, 2:])) + 2
            else:
                n = tableau[0, 2:].tolist().index(np.amin(tableau[0, 2:])) + 2

            # minimum ratio test, rth variable leaves basis 
            minimum = 99999
            r = -1

            for i in range(1, len(tableau)): 
                if(tableau[i, n] > 0):
                    val = tableau[i, 1]/tableau[i, n]
                    if val<minimum: 
                        minimum = val 
                        r = i
                            
            pivotElement = tableau[r, n] 
            
            if(self.printIter):
                print("Pivot Column:", n)
                print("Pivot Row:", r)
                print("Pivot Element: ", pivotElement)
                print("New Basis: " + str(tableau[r, 0]))

            for element in tableau[1:, n]:
                if (element != 0):
                    self.bounded = True
            
            if (not self.bounded):
                print("Unbounded; No solution.")
                return
            tableau = self.pivot(tableau, r, n)


            # new basic variable 
            tableau[r, 0] = n - 2
            
            
            iter += 1
            
        
        if(self.printIter == True):
            print("----------------------------------")
            print("Final Tableau reached in", iter, "iterations")
            self.printTableau(tableau)
        else:
            print("Solved")
            
        return tableau

In [212]:
model1 = LinearModel()

A = np.array([[1, 2, 3, 0], 
              [-1, 2, 6, 0], 
              [0, 4, 9, 0],
              [0, 0, 3, 1]])
b = np.array([3, 2, 5, 1])
c = np.array([1, 1, 1, 0])

model1.addA(A)
model1.addB(b)
model1.addC(c)
model1.setObj("MIN")
model1.setPrintIter(True)

print("A =\n", A, "\n")
print("b =\n", b, "\n")
print("c =\n", c, "\n\n")
model1.optimize()
print("\n")
model1.printSoln()

A =
 [[ 1  2  3  0]
 [-1  2  6  0]
 [ 0  4  9  0]
 [ 0  0  3  1]] 

b =
 [3 2 5 1] 

c =
 [1 1 1 0] 


Phase 1:

Starting Tableau:
ind 		x_0	x_1	x_2	x_3	s_0	s_1	s_2	s_3	
	-11.0	0.0	-8.0	-21.0	-1.0	0.0	0.0	0.0	0.0	
4	3.0	1.0	2.0	3.0	0.0	1.0	0.0	0.0	0.0	
5	2.0	-1.0	2.0	6.0	0.0	0.0	1.0	0.0	0.0	
6	5.0	0.0	4.0	9.0	0.0	0.0	0.0	1.0	0.0	
7	1.0	0.0	0.0	3.0	1.0	0.0	0.0	0.0	1.0	
----------------------------------
Iteration : 1
ind 		x_0	x_1	x_2	x_3	s_0	s_1	s_2	s_3	
	-11.0	0.0	-8.0	-21.0	-1.0	0.0	0.0	0.0	0.0	
4	3.0	1.0	2.0	3.0	0.0	1.0	0.0	0.0	0.0	
5	2.0	-1.0	2.0	6.0	0.0	0.0	1.0	0.0	0.0	
6	5.0	0.0	4.0	9.0	0.0	0.0	0.0	1.0	0.0	
7	1.0	0.0	0.0	3.0	1.0	0.0	0.0	0.0	1.0	
Pivot Column: 4
Pivot Row: 2
Pivot Element:  6.0
New Basis: 5.0
----------------------------------
Iteration : 2
ind 		x_0	x_1	x_2	x_3	s_0	s_1	s_2	s_3	
	-4.0	-3.5	-1.0	0.0	-1.0	0.0	3.5	0.0	0.0	
4	2.0	1.5	1.0	0.0	0.0	1.0	-0.5	0.0	0.0	
2	0.33	-0.17	0.33	1.0	0.0	0.0	0.17	0.0	0.0	
6	2.0	1.5	1.0	0.0	0.0	0.0	-1.5	1.0	0.0	
7	0.0	0.5	-1.0	0.0	1.0

In [213]:
A = np.array([[2, 1, -1, 0],
              [1, 7, 0, -1]])
b = np.array([4, 7])
c = np.array([1, 1, 0, 0])

model1.addA(A)
model1.addB(b)
model1.addC(c)
model1.setObj("MIN")
model1.setPrintIter(True)

print("A =\n", A, "\n")
print("b =\n", b, "\n")
print("c =\n", c, "\n\n")
model1.optimize()
print("\n")
model1.printSoln()

A =
 [[ 2  1 -1  0]
 [ 1  7  0 -1]] 

b =
 [4 7] 

c =
 [1 1 0 0] 


Phase 1:

Starting Tableau:
ind 		x_0	x_1	x_2	x_3	s_0	s_1	
	-11.0	-3.0	-8.0	1.0	1.0	0.0	0.0	
4	4.0	2.0	1.0	-1.0	0.0	1.0	0.0	
5	7.0	1.0	7.0	0.0	-1.0	0.0	1.0	
----------------------------------
Iteration : 1
ind 		x_0	x_1	x_2	x_3	s_0	s_1	
	-11.0	-3.0	-8.0	1.0	1.0	0.0	0.0	
4	4.0	2.0	1.0	-1.0	0.0	1.0	0.0	
5	7.0	1.0	7.0	0.0	-1.0	0.0	1.0	
Pivot Column: 3
Pivot Row: 2
Pivot Element:  7.0
New Basis: 5.0
----------------------------------
Iteration : 2
ind 		x_0	x_1	x_2	x_3	s_0	s_1	
	-3.0	-1.86	0.0	1.0	-0.14	0.0	1.14	
4	3.0	1.86	0.0	-1.0	0.14	1.0	-0.14	
1	1.0	0.14	1.0	0.0	-0.14	0.0	0.14	
Pivot Column: 2
Pivot Row: 1
Pivot Element:  1.8571428571428572
New Basis: 4.0
----------------------------------
Iteration : 3
ind 		x_0	x_1	x_2	x_3	s_0	s_1	
	0.0	0.0	0.0	0.0	0.0	1.0	1.0	
0	1.62	1.0	0.0	-0.54	0.08	0.54	-0.08	
1	0.77	0.0	1.0	0.08	-0.15	-0.08	0.15	
----------------------------------
Final Tableau reached in 3 iterations
ind 		x